In [1]:
import numpy as np

from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.factorization import explicit, implicit
from spotlight.evaluation import mrr_score, rmse_score
from collections import Counter
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import numpy as np

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
already_expected=True
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
if already_expected:
    df=pd.read_pickle('../reviews_Kindle_Store_10.h5')
    Training_DataFrame = pd.read_pickle('../reviews_Kindle_Store_10_training.h5')
    Testing_DataFrame = pd.read_pickle('../reviews_Kindle_Store_10_testing.h5')
else:
    df = getDF('../reviews_Kindle_Store_10.json.gz')
    


In [3]:

# Lists of unique movies and users
movie_list = list(set(df['asin'].values))
user_list = list(set(df['reviewerID'].values))

#Storing the indices of users to generate the Rating Memory Matrix
UserIndices={}
for i in range(len(user_list)):
    UserIndices[user_list[i]] = i

#Storing the indices of movies to generate the Rating Memory Matrix
#These indices will be used to populate the UsersRatingMemory for Memory based Collaborative Filtering
MovieIndices={}
for j in range(len(movie_list)):
    MovieIndices[movie_list[j]] = j

In [5]:
user_ids_tr=[]
item_ids_tr=[]
ratings_tr=[]
for i in range(0, len(Training_DataFrame)):
    item_id=Training_DataFrame.iloc[i]['asin']
    user_id=Training_DataFrame.iloc[i]['reviewerID']
    user_ind=UserIndices[user_id]
    item_ind=MovieIndices[item_id]
    user_ids_tr.append(user_ind)
    item_ids_tr.append(item_ind)
    ratings_tr.append(float(df.iloc[i]['overall']))

In [6]:
from spotlight.interactions import Interactions
user_ids_list_tr=np.array(user_ids_tr,dtype=np.int32)
item_ids_list_tr=np.array(item_ids_tr,dtype=np.int32)
ratings_list_tr=np.array(ratings_tr,dtype=np.float32)
dataset_tr = Interactions(user_ids_list_tr, item_ids_list_tr,ratings_list_tr)

In [ ]:
user_ids_tn=[]
item_ids_tn=[]
ratings_tn=[]
for i in range(0, len(Testing_DataFrame)):
    item_id=Testing_DataFrame.iloc[i]['asin']
    user_id=Testing_DataFrame.iloc[i]['reviewerID']
    user_ind=UserIndices[user_id]
    item_ind=itemIndices[item_id]
    user_ids_tn.append(user_ind)
    item_ids_tn.append(item_ind)
    ratings_tn.append(float(df.iloc[i]['overall']))

In [ ]:
from spotlight.interactions import Interactions
user_ids_list_tn=np.array(user_ids_tn,dtype=np.int32)
item_ids_list_tn=np.array(item_ids_tn,dtype=np.int32)
ratings_list_tn=np.array(ratings_tn,dtype=np.float32)
dataset_tn = Interactions(user_ids_list_tn, item_ids_list_tn,ratings_list_tn)

In [30]:
RANDOM_SEED = 42
LATENT_DIM = 32
NUM_EPOCHS = 10
BATCH_SIZE = 256
L2 = 1e-6
LEARNING_RATE = 1e-3

In [31]:
explicit_model = explicit.ExplicitFactorizationModel(loss='regression',
                                                     embedding_dim=LATENT_DIM,
                                                     n_iter=NUM_EPOCHS,
                                                     learning_rate=LEARNING_RATE,
                                                     batch_size=BATCH_SIZE,
                                                     l2=L2,
                                                     random_state=np.random.RandomState(RANDOM_SEED))
implicit_model = implicit.ImplicitFactorizationModel(loss='bpr',
                                                     embedding_dim=LATENT_DIM,
                                                     n_iter=NUM_EPOCHS,
                                                     learning_rate=LEARNING_RATE,
                                                     batch_size=BATCH_SIZE,
                                                     l2=L2,
                                                     random_state=np.random.RandomState(RANDOM_SEED))

In [34]:
implicit_model.fit(dataset_tr)

In [35]:
print('Implicit MRR: {:.2f}'.format(mrr_score(implicit_model, dataset_tn, train=dataset_tr).mean()))

Implicit MRR: 0.02


In [ ]:
explicit_model.fit(dataset_tr)

In [33]:
print('Implicit MRR: {:.2f}'.format(mrr_score(explicit_model, dataset_tn, train=dataset_tr).mean()))

Implicit MRR: 0.01
